In [29]:
import os
import pandas as pd

def find_file(path, suffix):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith(suffix):
                result.append(os.path.join(root, name))
    return result
# find_file('/path/to/ESD/en', '.txt')
# len(find_file('/path/to/ESD/en', '.wav'))
wav_files = find_file('/path/to/ESD/en', '.wav')

wav_dict = {}
for wav_file in wav_files:
    wav_dict[wav_file.split('/')[-1][:-4]] = wav_file
# len(wav_dict)

17500

In [93]:
from utils.utils import load_filelist
import pandas as pd
import numpy as np

df_total = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
for file in find_file('/path/to/ESD/en', '.txt'):
    # print(file)
    filename_text_emtiontag = load_filelist(file, split='\t')
    df = pd.DataFrame(filename_text_emtiontag, columns=['filename', 'text', 'emotiontag'])
    df_total = pd.concat([df_total, df])
# df_total
emotion_tags = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
train_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
val_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])
test_df = pd.DataFrame(columns=['filename', 'text', 'emotiontag'])

for emotion_tag in emotion_tags:
    df_e = df_total[df_total['emotiontag'] == emotion_tag].to_numpy()
    # df_e random train:val:test 8:1:1
    np.random.seed(42)
    np.random.shuffle(df_e)
    df_tr = pd.DataFrame(df_e[:int(len(df_e)*0.8)], columns=['filename', 'text', 'emotiontag'])
    df_va = pd.DataFrame(df_e[int(len(df_e)*0.8):int(len(df_e)*0.9)], columns=['filename', 'text', 'emotiontag'])
    df_te = pd.DataFrame(df_e[int(len(df_e)*0.9):], columns=['filename', 'text', 'emotiontag'])
    train_df = pd.concat([train_df, df_tr])
    val_df = pd.concat([val_df, df_va])
    test_df = pd.concat([test_df, df_te])
    # break

# len(test_df), len(train_df), test_df

(1750,
 14000,
         filename                                      text emotiontag
 0    0013_000459                   If they mother knew it.      Angry
 1    0020_000366  Mister share man, I move for a division.      Angry
 2    0019_000366  Mister share man, I move for a division.      Angry
 3    0020_000478        Put that rude grasshopper in jail!      Angry
 4    0019_000608     Mum shuts one's mouth up, doesn't it.      Angry
 ..           ...                                       ...        ...
 345  0013_001446   They'd never know I'd regular ran away.   Surprise
 346  0013_001481     And I never had a whooping cough why.   Surprise
 347  0013_001645               Do you think it's a woozle?   Surprise
 348  0011_001561                 That's a full grown colt.   Surprise
 349  0019_001425        She is now choosing skirt to wear.   Surprise
 
 [1750 rows x 3 columns])

In [98]:
# create filelist
train_val_test = "train"
df_w = train_df
new_filelist = f"/path/to/filelist.txt"
with open(new_filelist, "w", encoding="utf-8") as f:
    for index, row in df_w.iterrows():
        wav_path = wav_dict[row['filename']]
        sid = int(row['filename'].split('_')[0]) - 10
        text = row['text']
        efeature_path = '/path/to/mmefeature/' + row['emotiontag'].lower() + '.pt'
        # print(wav_path, sid, text, efeature_path)
        # break
        f.write(f"{wav_path}|{str(sid)}|{text}|{efeature_path}\n")